# Visualising the Current Status of COVID-19 Vaccination

## Introduction

This notebook performs exploratory data analysis on multiple datasets and visualises the consolidated data.

The main objective is to visualise COVID-19 vaccination process globally and monitor its progress.

Plotly is used to visualise the data.

Used datasets are:

- COVID-19 World Vaccination Progress
- Population by Country - 2020

Visualisations answer three simple questions:

- What vaccines are used and in which countries?
- What country is vaccinated more people?
- What country is vaccinated a larger percent from its population?

## 1. Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import math
import chart_studio.plotly as py
import cufflinks as cf
import plotly
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
init_notebook_mode(connected=True)
cf.go_offline()

## 2. EDA on Vaccine Data Frame

### Read the vaccine data frame with necessary columns & parsing the date column

In [2]:
df_vac = pd.read_csv('Country_Vaccinations.csv',
                 usecols=["country", "iso_code", "date","people_vaccinated", "vaccines"],
                 parse_dates = ['date'])

### Inspect the vaccine data frame

In [3]:
df_vac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6186 entries, 0 to 6185
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   country            6186 non-null   object        
 1   iso_code           5826 non-null   object        
 2   date               6186 non-null   datetime64[ns]
 3   people_vaccinated  3542 non-null   float64       
 4   vaccines           6186 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 241.8+ KB


### See which rows do not have ISO Code

In [4]:
df_vac_na = df_vac[df_vac['iso_code'].isnull()]

df_vac_na['country'].unique()

array(['England', 'Northern Ireland', 'Scotland', 'Wales'], dtype=object)

### Drop the rows including the countries above since they are already in the United Kingdom

In [5]:
# This command will also drop the rows that has no value in their people vaccinated column
# Since this column is cumulative sum, we are going to use max() function to find our value in final data frame

df_vac = df_vac.dropna()
df_vac.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3280 entries, 0 to 6185
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   country            3280 non-null   object        
 1   iso_code           3280 non-null   object        
 2   date               3280 non-null   datetime64[ns]
 3   people_vaccinated  3280 non-null   float64       
 4   vaccines           3280 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 153.8+ KB


### Create a dataframe that gets the maximum value of total vaccinations and groups by country

In [6]:
df_vac = pd.DataFrame(df_vac.groupby(["country","iso_code",'vaccines'])["people_vaccinated"].max())

# Resetting indexes to use the data frame in merge function

df_vac.reset_index(level=0, inplace=True)
df_vac.reset_index(level=0, inplace=True)
df_vac.reset_index(level=0, inplace=True)

df_vac.head()

,vaccines,iso_code,country,people_vaccinated
0,Pfizer/BioNTech,ALB,Albania,6073.0
1,Pfizer/BioNTech,AND,Andorra,3650.0
2,Oxford/AstraZeneca,AGO,Angola,6169.0
3,Oxford/AstraZeneca,AIA,Anguilla,3929.0
4,"Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V",ARG,Argentina,1876113.0


In [7]:
df_vac.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   vaccines           115 non-null    object 
 1   iso_code           115 non-null    object 
 2   country            115 non-null    object 
 3   people_vaccinated  115 non-null    float64
dtypes: float64(1), object(3)
memory usage: 3.7+ KB


The vaccines data frame is ready for merge, now perform EDA on population data frame.

## 3. EDA on Population Data Frame

### Read the population data frame with necessary columns & parsing the date column

In [8]:
df_pop = pd.read_csv('Country_Population.csv')

df_pop.rename(columns={'Country (or dependency)': 'country'}, inplace=True)

df_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235 entries, 0 to 234
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   country            235 non-null    object 
 1   Population (2020)  235 non-null    int64  
 2   Yearly Change      235 non-null    object 
 3   Net Change         235 non-null    int64  
 4   Density (P/Km²)    235 non-null    int64  
 5   Land Area (Km²)    235 non-null    int64  
 6   Migrants (net)     201 non-null    float64
 7   Fert. Rate         235 non-null    object 
 8   Med. Age           235 non-null    object 
 9   Urban Pop %        235 non-null    object 
 10  World Share        235 non-null    object 
dtypes: float64(1), int64(4), object(6)
memory usage: 20.3+ KB


### Merge two data frames

In [9]:
df = pd.merge(df_vac, df_pop, how='left', on='country')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115 entries, 0 to 114
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   vaccines           115 non-null    object 
 1   iso_code           115 non-null    object 
 2   country            115 non-null    object 
 3   people_vaccinated  115 non-null    float64
 4   Population (2020)  110 non-null    float64
 5   Yearly Change      110 non-null    object 
 6   Net Change         110 non-null    float64
 7   Density (P/Km²)    110 non-null    float64
 8   Land Area (Km²)    110 non-null    float64
 9   Migrants (net)     98 non-null     float64
 10  Fert. Rate         110 non-null    object 
 11  Med. Age           110 non-null    object 
 12  Urban Pop %        110 non-null    object 
 13  World Share        110 non-null    object 
dtypes: float64(6), object(8)
memory usage: 13.5+ KB


### We have population missing for six different countries, let's have a look what they are

In [11]:
df_na = df[df['Population (2020)'].isnull()]

df_na['country'].unique()

array(["Cote d'Ivoire", 'Czechia', 'Guernsey', 'Jersey',
       'Turks and Caicos Islands'], dtype=object)

### Investigate the population data frame to see if we can find their equivalents

In [12]:
df_pop['country'].unique()

array(['China', 'India', 'United States', 'Indonesia', 'Pakistan',
       'Brazil', 'Nigeria', 'Bangladesh', 'Russia', 'Mexico', 'Japan',
       'Ethiopia', 'Philippines', 'Egypt', 'Vietnam', 'DR Congo',
       'Turkey', 'Iran', 'Germany', 'Thailand', 'United Kingdom',
       'France', 'Italy', 'Tanzania', 'South Africa', 'Myanmar', 'Kenya',
       'South Korea', 'Colombia', 'Spain', 'Uganda', 'Argentina',
       'Algeria', 'Sudan', 'Ukraine', 'Iraq', 'Afghanistan', 'Poland',
       'Canada', 'Morocco', 'Saudi Arabia', 'Uzbekistan', 'Peru',
       'Angola', 'Malaysia', 'Mozambique', 'Ghana', 'Yemen', 'Nepal',
       'Venezuela', 'Madagascar', 'Cameroon', "Côte d'Ivoire",
       'North Korea', 'Australia', 'Niger', 'Taiwan', 'Sri Lanka',
       'Burkina Faso', 'Mali', 'Romania', 'Malawi', 'Chile', 'Kazakhstan',
       'Zambia', 'Guatemala', 'Ecuador', 'Syria', 'Netherlands',
       'Senegal', 'Cambodia', 'Chad', 'Somalia', 'Zimbabwe', 'Guinea',
       'Rwanda', 'Benin', 'Burundi', 'Tuni

### Replace the countries in the population data frame with their corresponding name in the vaccine data frame, and follow the same steps

In [13]:
df_pop.country = df_pop.country.replace().replace({
    "Côte d'Ivoire": "Cote d'Ivoire", 
    "Czech Republic (Czechia)": "Czechia", 
    "Turks and Caicos": "Turks and Caicos Islands"})

df = pd.merge(df_vac, df_pop, how='left', on='country')

df_na = df[df['Population (2020)'].isnull()]

df_na['country'].unique()

array(['Guernsey', 'Jersey'], dtype=object)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115 entries, 0 to 114
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   vaccines           115 non-null    object 
 1   iso_code           115 non-null    object 
 2   country            115 non-null    object 
 3   people_vaccinated  115 non-null    float64
 4   Population (2020)  113 non-null    float64
 5   Yearly Change      113 non-null    object 
 6   Net Change         113 non-null    float64
 7   Density (P/Km²)    113 non-null    float64
 8   Land Area (Km²)    113 non-null    float64
 9   Migrants (net)     100 non-null    float64
 10  Fert. Rate         113 non-null    object 
 11  Med. Age           113 non-null    object 
 12  Urban Pop %        113 non-null    object 
 13  World Share        113 non-null    object 
dtypes: float64(6), object(8)
memory usage: 13.5+ KB


### Since we do not have these countries, we can drop the rows that include them

In [15]:
# Before dropping rows, drop the Migrants (net) column since it includes
# more missing data and dropping NaN values will cause loss of data

del df['Migrants (net)']

# Delete the Yearly Change, Fert. Rate, Med. Age, Urban Pop %, World Share columnssince they are not relevant

del df["Yearly Change"]
del df["Fert. Rate"]
del df["Med. Age"]
del df["Urban Pop %"]
del df["World Share"]

# Drop NaN columns one more time to remove the rows for Guernsey and Jersey

df_f = df.dropna()

df_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113 entries, 0 to 114
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   vaccines           113 non-null    object 
 1   iso_code           113 non-null    object 
 2   country            113 non-null    object 
 3   people_vaccinated  113 non-null    float64
 4   Population (2020)  113 non-null    float64
 5   Net Change         113 non-null    float64
 6   Density (P/Km²)    113 non-null    float64
 7   Land Area (Km²)    113 non-null    float64
dtypes: float64(5), object(3)
memory usage: 7.9+ KB


### Have a look at the final data frame

In [16]:
df_f = df_f.reset_index(drop=True)

df_f.head()

,vaccines,iso_code,country,people_vaccinated,Population (2020),Net Change,Density (P/Km²),Land Area (Km²)
0,Pfizer/BioNTech,ALB,Albania,6073.0,2877239.0,-3120.0,105.0,27400.0
1,Pfizer/BioNTech,AND,Andorra,3650.0,77287.0,123.0,164.0,470.0
2,Oxford/AstraZeneca,AGO,Angola,6169.0,33032075.0,1040977.0,26.0,1246700.0
3,Oxford/AstraZeneca,AIA,Anguilla,3929.0,15026.0,134.0,167.0,90.0
4,"Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V",ARG,Argentina,1876113.0,45267449.0,415097.0,17.0,2736690.0


## 4. Creating a Detailed Vaccine Data Frame

### Have a look at the vaccines column to see how many unique vaccine groups are present

In [17]:
df_f['vaccines'].unique()

array(['Pfizer/BioNTech', 'Oxford/AstraZeneca',
       'Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V',
       'Oxford/AstraZeneca, Pfizer/BioNTech',
       'Moderna, Oxford/AstraZeneca, Pfizer/BioNTech', 'Sinovac',
       'Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V',
       'Sputnik V', 'Oxford/AstraZeneca, Sinovac', 'Sinopharm/Beijing',
       'Moderna, Pfizer/BioNTech', 'Pfizer/BioNTech, Sinovac', 'Moderna',
       'Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V',
       'Covaxin, Oxford/AstraZeneca',
       'Pfizer/BioNTech, Sinopharm/Beijing',
       'Pfizer/ BioNTech, Sinopharm/Beijing',
       'Oxford/AstraZeneca, Pfizer/BioNTech, Sputnik V',
       'Sinopharm/Beijing, Sputnik V',
       'Oxford/AstraZeneca, Sinopharm/Beijing', 'EpiVacCorona, Sputnik V',
       'Johnson&Johnson',
       'Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sinopharm/Wuhan, Sputnik V',
       'Johnson&Johnson, Moderna, Pfizer/BioNTech'], dt

### Create boolean columns for all vaccines and concatenate them in one data frame

In [18]:
boo1 = df_f['vaccines'].str.contains('Pfizer/BioNTech')
df_boo1 = pd.DataFrame(boo1).reset_index(drop=True)
df_boo1.rename(columns={'vaccines': 'Pfizer/BioNTech'}, inplace=True)

boo2 = df_f['vaccines'].str.contains('Sputnik V')
df_boo2 = pd.DataFrame(boo2).reset_index(drop=True)
df_boo2.rename(columns={'vaccines': 'Sputnik V'}, inplace=True)

boo3 = df_f['vaccines'].str.contains('Oxford/AstraZeneca')
df_boo3 = pd.DataFrame(boo3).reset_index(drop=True)
df_boo3.rename(columns={'vaccines': 'Oxford/AstraZeneca'}, inplace=True)

boo4 = df_f['vaccines'].str.contains('Sinopharm/Beijing')
df_boo4 = pd.DataFrame(boo4).reset_index(drop=True)
df_boo4.rename(columns={'vaccines': 'Sinopharm/Beijing'}, inplace=True)

boo5 = df_f['vaccines'].str.contains('Moderna')
df_boo5 = pd.DataFrame(boo5).reset_index(drop=True)
df_boo5.rename(columns={'vaccines': 'Moderna'}, inplace=True)

boo6 = df_f['vaccines'].str.contains('Sinovac')
df_boo6 = pd.DataFrame(boo6).reset_index(drop=True)
df_boo6.rename(columns={'vaccines': 'Sinovac'}, inplace=True)

boo7 = df_f['vaccines'].str.contains('Covaxin')
df_boo7 = pd.DataFrame(boo7).reset_index(drop=True)
df_boo7.rename(columns={'vaccines': 'Covaxin'}, inplace=True)

boo8 = df_f['vaccines'].str.contains('EpiVacCorona')
df_boo8 = pd.DataFrame(boo8).reset_index(drop=True)
df_boo8.rename(columns={'vaccines': 'EpiVacCorona'}, inplace=True)

boo9 = df_f['vaccines'].str.contains('Sinopharm/Wuhan')
df_boo9 = pd.DataFrame(boo9).reset_index(drop=True)
df_boo9.rename(columns={'vaccines': 'Sinopharm/Wuhan'}, inplace=True)

result = pd.concat([df_boo1, df_boo2, df_boo3, df_boo4, df_boo5, df_boo6, df_boo7, df_boo8, df_boo9], axis=1)

result = result.astype(int)

result.head()

,Pfizer/BioNTech,Sputnik V,Oxford/AstraZeneca,Sinopharm/Beijing,Moderna,Sinovac,Covaxin,EpiVacCorona,Sinopharm/Wuhan
0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0
4,0,1,1,1,0,0,0,0,0


### Merge detailed vaccine data frame to the main data frame and delete the vaccines column, then add a column for the total number of different vaccines

In [19]:
df_f = pd.concat([df_f, result], axis=1)

df_f['Different Vaccines'] = df_f['Pfizer/BioNTech'] + df_f['Sputnik V']+ df_f['Oxford/AstraZeneca']+ df_f['Sinopharm/Beijing']+ df_f['Moderna']+ df_f['Sinovac']+ df_f['Covaxin']+ df_f['EpiVacCorona']+ df_f['Sinopharm/Wuhan']

df_f.head()

,vaccines,iso_code,country,people_vaccinated,Population (2020),Net Change,Density (P/Km²),Land Area (Km²),Pfizer/BioNTech,Sputnik V,Oxford/AstraZeneca,Sinopharm/Beijing,Moderna,Sinovac,Covaxin,EpiVacCorona,Sinopharm/Wuhan,Different Vaccines
0,Pfizer/BioNTech,ALB,Albania,6073.0,2877239.0,-3120.0,105.0,27400.0,1,0,0,0,0,0,0,0,0,1
1,Pfizer/BioNTech,AND,Andorra,3650.0,77287.0,123.0,164.0,470.0,1,0,0,0,0,0,0,0,0,1
2,Oxford/AstraZeneca,AGO,Angola,6169.0,33032075.0,1040977.0,26.0,1246700.0,0,0,1,0,0,0,0,0,0,1
3,Oxford/AstraZeneca,AIA,Anguilla,3929.0,15026.0,134.0,167.0,90.0,0,0,1,0,0,0,0,0,0,1
4,"Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V",ARG,Argentina,1876113.0,45267449.0,415097.0,17.0,2736690.0,0,1,1,1,0,0,0,0,0,3


Now we have our final data frame with no missing values, so we can start visualizing the data.

## 5. Data Visualisation

### Which vaccine is most commonly used around the world?

In [20]:
# Create a data frame to calculate the usage of every individual vaccine in different countries

summ = df_f.iloc[:,8:17].sum()
df_vacc = pd.DataFrame(summ, columns=['Usage'])
df_vacc = df_vacc.sort_values('Usage', ascending=False)
df_vacc.reset_index(level=0, inplace=True)

# Visualise the vaccine usage

fig = px.bar(df_vacc,
              x = 'index',
              y = 'Usage',
              color='Usage',
              hover_name = 'index',
              hover_data = ['Usage'],
              color_continuous_scale = 'Magenta',
              labels = {'index':'Vaccine','Usage':'# of Countries Using'},
              height=500,
              text = 'Usage')

fig.update_layout(uniformtext_minsize = 15,
                   xaxis_tickangle = -45,
                   title = 'Usage of Vaccines by Countries',
                   title_x = 0.5,
                   showlegend=False)

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

fig.update(layout_coloraxis_showscale=False)
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)', 'paper_bgcolor': 'rgba(0, 0, 0, 0)',})

fig.show()

### Vaccination Race

It appears that the vaccine developed by Pfizer/Biontech benefits being first in the market. Although the vaccine developed by Oxford/AstraZeneca was released some time after the one developed by Pfizer/Biontech, it is a strong competent in the vaccination race since it is easier to roll out in terms of handling and transportation. Currently These two are the most common COVID-19 vaccines around the world by stepping in more than 60 countries.

### What vaccines are used and in which countries?

In [21]:
df_f['Log Scale'] = df_f['people_vaccinated'].apply(lambda x : math.log2(x+1))

df_f_sorted = df_f.sort_values('Different Vaccines', ascending = False)

fig = px.bar(df_f_sorted,
              x = 'country',
              y = 'Different Vaccines',
              color='Different Vaccines',
              hover_name = 'country',
              hover_data = ['vaccines'],
              color_continuous_scale = 'Blues',
              labels = {'country':'Country', 'vaccines':'Used Vaccines'},  # changing the labels in to tooltip
              height=500)

fig.update_layout(uniformtext_minsize = 15,
                   xaxis_tickangle = -45,
                   title = 'Total Number of Different Vaccines Used by Countries',
                   title_x = 0.5)

fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)', 'paper_bgcolor': 'rgba(0, 0, 0, 0)',})

fig.update(layout_coloraxis_showscale=False)

fig.show()

### Does more money mean more different types of vaccine?

It looks like UAE and Hungary are the countries who uses 5 of 9 different kind of vaccines. They are followed by Bahrain and Serbia with 4 different kind of vaccines used. Let's have a look at the distribution in the map to understand it better.

In [22]:
fig = px.choropleth(df_f,
                    locations="country", 
                    locationmode='country names',
                    color="Different Vaccines", 
                    hover_name="country", 
                    hover_data=['Different Vaccines','vaccines'],
                    color_continuous_scale="Blues",
                    labels={'country':'Country','vaccines':'Used Vaccines'})

fig.update_layout(title="Total Number of Different Vaccines Used by Countries",
                  title_x=0.5)

fig.update(layout_coloraxis_showscale=False)

fig.show()

### What country is vaccinated more people?

In [23]:
# Sort data frame by total_vaccinations
df_f_sorted = df_f.sort_values('people_vaccinated', ascending = False)

# Select first 10 rows
df_f_sorted = df_f_sorted.iloc[0:19,:]

# Plot the bar chart
fig = px.bar(df_f_sorted,
              x = 'country',
              y = 'people_vaccinated',
              color='Log Scale',
              hover_name = 'country',
              hover_data = ['people_vaccinated'],
              color_continuous_scale = 'mint',
              labels = {'country':'Country','people_vaccinated':'People Vaccinated'},
              height=500,
              text='people_vaccinated')

fig.update_layout(uniformtext_minsize = 15,
                   xaxis_tickangle = -45,
                   title = 'Total Vaccinated People by Top 20 Countries',
                   title_x = 0.5)

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)', 'paper_bgcolor': 'rgba(0, 0, 0, 0)',})

fig.update(layout_coloraxis_showscale=False)

fig.show()

### More population, more people to vaccinate

United States are far above most of the countries with almost 70M people vaccinated. It is followed by India, UK, Brazil and Turkey. Given most of these countries have the highest population in the world, it is not suprising they have higher number of people vaccinated compared to the rest.

In [24]:
fig = px.choropleth(df_f,
                    locations="country", 
                    locationmode='country names',
                    color="Log Scale", 
                    hover_name="country", 
                    hover_data=['people_vaccinated'],
                    color_continuous_scale="mint",
                    labels={'country':'Country','people_vaccinated':'People Vaccinated'})

fig.update_layout(title="Total Vaccinated People by Country", title_x=0.5)

fig.update(layout_coloraxis_showscale=False)

fig.show()

### Numbers are definitely not wrong, but might be misleading

It is clear that the countries with high number of population has also high number of people vaccinated. To understand the countries' success in vaccination process better, we should look at the % of the population vaccinated by country.

### What country is vaccinated a larger percent from its population?

In [25]:
df_f['pct_vaccinated'] = df_f['people_vaccinated'] / df_f['Population (2020)']

# Sort data frame by pct_vaccinated
df_f_sorted = df_f.sort_values('pct_vaccinated', ascending = False)

# Select first 10 rows
df_f_sorted = df_f_sorted.iloc[0:19,:]

# Plot the bar chart
fig = px.bar(df_f_sorted,
              x = 'country',
              y = 'pct_vaccinated',
              color='pct_vaccinated',
              hover_name = 'country',
              hover_data = ['pct_vaccinated'],
              color_continuous_scale = 'Peach',
              labels = {'country':'Country','pct_vaccinated':'% of Population Vaccinated'},
              height=500)

fig.update_layout(uniformtext_minsize = 15,
                   xaxis_tickangle = -45,
                   title = 'Total Vaccination % of Population by Top 20 Countries',
                   title_x = 0.5)

fig.update(layout_coloraxis_showscale=False)
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)', 'paper_bgcolor': 'rgba(0, 0, 0, 0)',})

fig.show()

### Surprise surprise

Let's be honest, no one is expecting Gibraltar or Seychelles to be the first in the vaccinated people % of the population, yet, here it is...

The final results shows the top 5 best performing countries in the vaccination process, and they are

- Gibraltar
- Seychelles
- Israel
- Falkland Islands
- United Arab Emirates



In [26]:
fig = px.choropleth(df_f,
                    locations="country", 
                    locationmode='country names',
                    color="pct_vaccinated", 
                    hover_name="country", 
                    hover_data=['pct_vaccinated'],
                    color_continuous_scale="Peach",
                    labels={'country':'Country','pct_vaccinated':'% of Population Vaccinated'})

fig.update_layout(title="Total Vaccination % of Population by Country", title_x=0.5)

fig.update(layout_coloraxis_showscale=False)

fig.show()

## 6. Conclusion

### Relativity is everything

Considering every single country is responsible for their own people (apart from minority groups such as immigrants), we can understand that the success in vaccination process should not only be assessed with the number of people vaccinated. When we look at the total vaccination % of the population, we see a more accurate indicator of success and understand that the measure of success is quality rather than quantity.

## Before closing...

### If you've made it so far in the notebook, first, congratulations! and second, please let me know about your thoughts!

- LinkedIn: https://www.linkedin.com/in/emrecanokten/
- Kaggle:https://www.kaggle.com/emrecano

### And do not forget the vote the notebook or like the post (whichever is applicable)!

# Thanks a lot!